# Multi-agent preparation for job application and interview
This example demonstrates a Semantic Kernel Agents solution to: 
- upload a candidate resume (or use a mock)
- scrape an open job position from a web url (or use a mock)
- update a candidate resume to target it to the open job position
- prepare a set of interview questions to support the candidate


## Imports and Setup
First, import the necessary libraries and do some setup

In [ ]:
import os
import pathlib
import logging
import yaml
import requests
import requests
import logging
import re
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from typing import Annotated

from semantic_kernel.kernel import Kernel
from semantic_kernel.contents import ChatHistoryTruncationReducer
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions import kernel_function, KernelArguments, KernelPlugin, KernelFunctionFromPrompt
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.agents.strategies import KernelFunctionSelectionStrategy, KernelFunctionTerminationStrategy
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.connectors.ai.azure_ai_inference import AzureAIInferenceChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

### Logging Configuration

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Set higher logging level for the azure libraries to suppress verbose HTTP logs, so we can focus on Semantic Kernel logs
logging.getLogger("azure").setLevel(logging.WARNING)
logging.getLogger("azure.core.pipeline.policies.http_logging_policy").setLevel(logging.WARNING)

### Environment Variables

In [ ]:
# Load environment variables from .env file
# Look for .env in the current directory and parent directory
current_dir = pathlib.Path().absolute()
root_dir = current_dir.parent
load_dotenv(dotenv_path=root_dir / ".env")

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")

### LLM Model Service

In [ ]:
GPT4O_SERVICE = AzureAIInferenceChatCompletion(
    ai_model_id="gpt-4o",
    endpoint=f"{str(AZURE_OPENAI_ENDPOINT).strip('/')}/openai/deployments/{AZURE_OPENAI_DEPLOYMENT_NAME}",
    api_key=AZURE_OPENAI_API_KEY,
)

### Semantic Kernel Configuration

In [ ]:
WEB_JOB_RESEARCH_AGENT_NAME = "Web-Job-Research-Agent"
RESUME_COPYWRITER_AGENT_NAME = "Resume-Copywriter-Agent"
RESUME_REVIEWER_AGENT_NAME = "Resume-Reviewer-Agent"
INTERVIEW_PREPARATION_AGENT_NAME = "Interview-Preparation-Agent"

RESUME_REVIEW_CONTINUE_KEYWORD = "RESUME_REVIEW_CONTINUE"
RESUME_REVIEW_COMPLETE_KEYWORD = "RESUME_REVIEW_COMPLETE"
INTERVIEW_PREP_NEEDED = "INTERVIEW_PREP_NEEDED"
TERMINATION_KEYWORD = "PROCESS_COMPLETE"
PROCESS_COMPLETE = "COMPLETE"

MAXIMUM_CHAT_ITERATIONS=12
MAXIMUM_HISTORY_MESSAGES=3

### Data Sources Configuration

In [ ]:
USE_MOCK_JOB_POSTING = True
MOCK_JOB_POSTING_PATH = "09_5_mock_job_posting.txt"
JOB_POSTING_URL = "https://jobs.lever.co/AIFund/29e4750a-61c1-4195-9a11-7889577e3d6f"
JOB_POSTING_MAX_LENGTH = 8000  # Adjust based on LLM's context window size

RESUME_PATH = "09_5_mock_resume.txt"

## Configure 3 Pillars of Observability

In [ ]:
from opentelemetry.sdk.resources import Resource
from opentelemetry.semconv.resource import ResourceAttributes
from opentelemetry._logs import set_logger_provider
from opentelemetry.metrics import set_meter_provider
from opentelemetry.trace import set_tracer_provider, get_tracer

from opentelemetry.sdk.trace import TracerProvider, ReadableSpan
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.sdk.metrics import MeterProvider
from opentelemetry.sdk.metrics.export import PeriodicExportingMetricReader
from opentelemetry.sdk.metrics.view import DropAggregation, View
from opentelemetry.sdk._logs import LoggerProvider, LoggingHandler
from opentelemetry.sdk._logs.export import BatchLogRecordProcessor

from azure.monitor.opentelemetry.exporter import (
    AzureMonitorLogExporter,
    AzureMonitorMetricExporter,
    AzureMonitorTraceExporter,
)

In [ ]:
class CustomSpanProcessor(BatchSpanProcessor):
    """Filtering out spans with specific names and URLs to keep only Semantic Kernel telemetry"""

    EXCLUDED_SPAN_NAMES = ['.*CosmosClient.*', '.*DatabaseProxy.*', '.*ContainerProxy.*']

    def on_end(self, span: ReadableSpan) -> None:
       
        for regex in self.EXCLUDED_SPAN_NAMES:
            if re.match(regex, span.name):
                return
            
        if span.attributes.get('component') == 'http':
            return
    
        super().on_end(span)

### Set up tracing
To inspect telemetry data, navigate to your Application Insights resource in the Azure portal, then navigate to the **Transactions search** tab to view the traced transactions, once the agent workload has completed. For more info, see:

https://learn.microsoft.com/en-us/semantic-kernel/concepts/enterprise-readiness/observability/telemetry-with-app-insights?tabs=Powershell&pivots=programming-language-python

In [ ]:
# Set up tracing
exporters = []
exporters.append(AzureMonitorTraceExporter.from_connection_string(os.getenv("APPLICATIONINSIGHTS_CONNECTION_STRING")))
telemetry_resource = Resource.create({ResourceAttributes.SERVICE_NAME: os.getenv("AZURE_RESOURCE_GROUP","rg-ai-services-core")})

tracer_provider = TracerProvider(resource=telemetry_resource)
for trace_exporter in exporters:
    tracer_provider.add_span_processor(CustomSpanProcessor(trace_exporter))
set_tracer_provider(tracer_provider)

### Set up metrics


In [ ]:
exporters = []
exporters.append(AzureMonitorMetricExporter.from_connection_string(os.getenv("APPLICATIONINSIGHTS_CONNECTION_STRING")))

metric_readers = [PeriodicExportingMetricReader(exporter, export_interval_millis=5000) for exporter in exporters]

meter_provider = MeterProvider(
    metric_readers=metric_readers,
    resource=telemetry_resource,
    views=[
        # Dropping all instrument names except for those starting with "semantic_kernel"
        View(instrument_name="*", aggregation=DropAggregation()),
        View(instrument_name="semantic_kernel*"),
    ],
)
set_meter_provider(meter_provider)

In [ ]:
exporters = []
exporters.append(AzureMonitorLogExporter(connection_string=os.getenv("APPLICATIONINSIGHTS_CONNECTION_STRING")))


logger_provider = LoggerProvider(resource=telemetry_resource)
set_logger_provider(logger_provider)

handler = LoggingHandler()

logger = logging.getLogger()
logger.addHandler(handler)
logger.setLevel(logging.INFO)

for log_exporter in exporters:
    logger_provider.add_log_record_processor(BatchLogRecordProcessor(log_exporter))

# FILTER - WHAT NOT TO LOG
class KernelFilter(logging.Filter):
    """A filter to not process records from semantic_kernel."""

    # These are the namespaces that we want to exclude from logging for the purposes of this notebook
    namespaces_to_exclude: list[str] = [
        # "semantic_kernel.functions.kernel_plugin",
        "semantic_kernel.prompt_template.kernel_prompt_template",
        # "semantic_kernel.functions.kernel_function",
        "azure.monitor.opentelemetry.exporter.export._base",
        "azure.core.pipeline.policies.http_logging_policy"
    ]

    def filter(self, record):
        return not any([record.name.startswith(namespace) for namespace in self.namespaces_to_exclude])

handler.addFilter(KernelFilter())

## Define our individual agents
Structured defintion of our agent personas that charcterises how the agents should operate and interact.

In [ ]:
web_job_research_agent_persona = """
name: "Web-Job-Research-Agent"
description: Tech job posting analyzer that extracts key requirements for applicants.
temperature: 0.1
included_plugins: []
instructions: |
  Extract essential information from job postings to help with application preparation:
    1. Scrape the provided job posting URL using scrape_website tool
    2. Identify and categorize requirements into:
    - Technical Skills (languages, tools, platforms)
    - Soft Skills (communication, teamwork)
    - Experience (years, specific domains)
    - Education (degrees, certifications)
    - Company Values/Culture
    3. Highlight any must-have qualifications vs. preferred or nice to have qualifications
    Focus on extracting actionable information that helps tailor resumes.
"""

In [ ]:
resume_reviewer_persona = f"""
name: "Resume-Reviewer-Agent"
description: Analyzes resume-job fit and provides targeted improvement recommendations.
temperature: 0.1
included_plugins: []
instructions: |
  Evaluate how well a candidate's resume aligns with job requirements from {WEB_JOB_RESEARCH_AGENT_NAME}:
  
  1. First, you will load the resume.
  1. Review the uploaded resume against the job posting summary
  2. Identify gaps between resume content and job requirements
  3. Provide concise, specific, actionable recommendations to improve:
     - Skills alignment (missing technical/soft skills)
     - Experience presentation (achievements, metrics, relevance)
     - Keywords/terminology matching
     - Overall format and impact
  4. Track previously suggested changes - never repeat recommendations
  5. Verify if prior suggestions were implemented
  6. Only suggest changes grounded in both the resume and job requirements
  7. Do not make direct edits to the resume
  8. When no further improvements needed, respond only with: {RESUME_REVIEW_COMPLETE_KEYWORD}
"""

In [ ]:
resume_copywriter_persona = f"""
name: "Resume-Copywriter-Agent"
description: Implements targeted resume improvements based on reviewer recommendations.
temperature: 0.1
included_plugins: []
instructions: |
  Update candidate resumes based on expert recommendations:
  
  1. Review improvement suggestions from {RESUME_REVIEWER_AGENT_NAME}
  2. Implement changes to the resume focusing on:
     - Enhancing relevance to job requirements
     - Highlighting transferable skills
     - Strengthening achievement statements with metrics
     - Incorporating job-specific keywords
     - Improving clarity and impact
  3. Ground your updates on the candidate's original experience and qualifications
  4. Preserve the resume's formatting structure
  5. Return the complete updated resume with changes implemented
  
  Your goal is to transform the resume to maximize the candidate's chances of passing automated screening and impressing human reviewers.
"""


In [ ]:
interview_preparation_persona = f"""
name: "Interview-Preparation-Agent"
description: Create interview questions and talking points based on updated resume and job requirements
temperature: 0.2
included_plugins: []
instructions: |
  Your role is to prepare candidates to ensure they can confidently address all aspects of the job they are applying for by:
  
  1. Analyzing both the job posting requirements and the candidate's final updated resume deemed {RESUME_REVIEW_COMPLETE_KEYWORD} by {RESUME_REVIEWER_AGENT_NAME}
  2. Create likely technical interview questions based on the job posting's requirements
  3. Develop talking points that highlight how the candidate's experience aligns with job requirements
  4. Prepare candidate responses for potential questions about gaps or missing requirements
  5. Formulate examples of behavioral questions specific to the role
  6. Suggest discussion points about company culture and values
"""

In [ ]:
class WebJobResearchAgentPlugin:
    def __init__(self):
        pass  

    @kernel_function(
        description="Fetch a job posting from the Web using a url", 
    )
    async def fetch_job_posting(self, url:Annotated[str,"The url of the job posting"]) -> Annotated[str, "The output in str format"]:
        """
        Fetch the job posting from the provided URL and analyze it to extract key skills, experiences, and qualifications required.

        Parameters:
        url (str): The URL of the job posting to analyze.

        Returns:
        str: A structured list of job requirements, including necessary skills, qualifications, and experiences.
        """
        if USE_MOCK_JOB_POSTING:
            try:
                with open(MOCK_JOB_POSTING_PATH, 'r', encoding='utf-8') as file:
                    content = file.read()
                logging.info(f"Successfully loaded mock job posting from {MOCK_JOB_POSTING_PATH}. Length: {len(content)} characters")
                               
                if len(content) > JOB_POSTING_MAX_LENGTH:
                    logging.info(f"Truncating mock content from {len(content)} to {JOB_POSTING_MAX_LENGTH} characters")
                    content = content[:JOB_POSTING_MAX_LENGTH]
                
                return content
            except Exception as e:
                logging.error(f"Failed to load mock job posting: {str(e)}")
                # Fall back to web scraping if mock fails
                pass
            
        # Continue with web scraping if not using mock or if mock loading failed
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
                "Accept-Language": "en-US,en;q=0.5",
                "Connection": "keep-alive",
                "Upgrade-Insecure-Requests": "1",
                "Cache-Control": "max-age=0"
            }
        
            # Fetch the HTML content with a timeout
            response = requests.get(url, headers=headers, timeout=30)
            response.raise_for_status()
            
            html_content = response.text
            logging.info(f"Fetched {len(html_content)} characters from {url}")
            
            # Now we want to parse the HTML to remove unnecessary elements, which save us a huge amount of LLM tokens
            soup = BeautifulSoup(html_content, 'html.parser')
            
            # Remove script, style, and other non-content elements
            for element in soup(['script', 'style', 'head', 'meta', 'link', 'noscript', 'iframe']):
                element.decompose()
            
            # Get all text and do further cleaning
            text = soup.get_text(separator='\n', strip=True)
            text = re.sub(r'\n\s*\n', '\n\n', text)
            text = re.sub(r' +', ' ', text)
            
            # Remove very short lines that might be menu items or UI elements
            lines = text.split('\n')
            filtered_lines = [line for line in lines if len(line.strip()) > 3]
            text = '\n'.join(filtered_lines)
            
            # Add URL reference
            text = f"Job posting from: {url}\n\n{text}"

            if len(text) > JOB_POSTING_MAX_LENGTH:
                logging.info(f"Truncating parsed content from {len(text)} to {JOB_POSTING_MAX_LENGTH} characters")
                text = text[:JOB_POSTING_MAX_LENGTH]
            
            logging.info(f"Successfully extracted content. Length: {len(text)} characters")
            return text
        except:
            logging.error(f"Failed to fetch the job posting from {url}.")
            return 

In [ ]:
class ResumeReviewerAgentPlugin:
    def __init__(self):
        pass

    @kernel_function(
        description="Load a resume either from a provided file path or use a mock resume if no path is provided",
    )
    async def load_resume(self, file_path: Annotated[str, "Optional path to a resume file"] = "") -> Annotated[str, "The resume content"]:
        """
        Load a resume either from a provided file path or use a mock resume if no path is provided.

        Parameters:
        file_path (str): Optional path to a resume file. If empty, a mock resume will be used.

        Returns:
        str: The content of the resume.
        """
        try:
            if RESUME_PATH and RESUME_PATH.strip():
                with open(RESUME_PATH, 'r') as file:
                    resume_content = file.read()
                logging.info(f"Successfully loaded resume from {file_path}")
                return resume_content
        except Exception as e:
            logging.error(f"Failed to load resume: {str(e)}")
            return f"Error loading resume: {str(e)}"

In [ ]:
class ResumeCopywriterAgentPlugin:
    def __init__(self):
        pass
           

In [ ]:
class InterviewPreparationAgentPlugin:
    def __init__(self):
        pass

## Helper functions

In [ ]:
def create_agent(kernel, service_id, definition):

    definition = yaml.safe_load(definition)
    execution_settings=AzureChatPromptExecutionSettings(
            temperature=definition.get('temperature', 0.5),
            function_choice_behavior=FunctionChoiceBehavior.Auto(
                filters={"included_plugins": definition.get('included_plugins', [])}
            )
        )
    
    return ChatCompletionAgent(
        service=kernel.get_service(service_id=service_id),
        kernel=kernel,
        arguments=KernelArguments(settings=execution_settings),
        name=definition['name'],
        description=definition['description'],
        instructions=definition['instructions']
    )

In [ ]:
kernel = Kernel(
    services=[GPT4O_SERVICE],
    plugins=[
        KernelPlugin.from_object(plugin_instance=WebJobResearchAgentPlugin(), plugin_name="WebJobResearchAgent"),
        KernelPlugin.from_object(plugin_instance=ResumeReviewerAgentPlugin(), plugin_name="ResumeReviewerAgent"),
        ],
)

## Create agent group chat

### Create the agents

In [ ]:
web_job_research_agent = create_agent(service_id="gpt-4o",
                                      kernel=kernel,
                                      definition=web_job_research_agent_persona)

resume_copywriter_agent = create_agent(service_id="gpt-4o",
                                       kernel=kernel,
                                       definition=resume_copywriter_persona)

resume_reviewer_agent = create_agent(service_id="gpt-4o",
                                     kernel=kernel,
                                     definition=resume_reviewer_persona)

interview_preparer_agent = create_agent(service_id="gpt-4o",
                                        kernel=kernel,
                                        definition=interview_preparation_persona)


### Define which of the agents responds next in the conversation

In [ ]:
selection_function = KernelFunctionFromPrompt(
        function_name="selection",
        prompt=f"""
        Examine the provided RESPONSE and choose the next participant.
        State only the name of the chosen participant without explanation.
        Never choose the participant named in the RESPONSE.

        Choose only from these participants:
        - {WEB_JOB_RESEARCH_AGENT_NAME}
        - {RESUME_REVIEWER_AGENT_NAME} 
        - {RESUME_COPYWRITER_AGENT_NAME}
        - {INTERVIEW_PREPARATION_AGENT_NAME}

        Abide by the following policy:
        - If RESPONSE is user, it is {WEB_JOB_RESEARCH_AGENT_NAME}'s turn. It's VERY IMPORTANT you only choose {WEB_JOB_RESEARCH_AGENT_NAME} one time!
        - If RESPONSE is by {WEB_JOB_RESEARCH_AGENT_NAME}, it is {RESUME_REVIEWER_AGENT_NAME}'s turn. It's VERY IMPORTANT there is a review by {RESUME_REVIEWER_AGENT_NAME} at least once.
        - If RESPONSE is by {RESUME_REVIEWER_AGENT_NAME}, it is {RESUME_COPYWRITER_AGENT_NAME}'s turn. It's VERY IMPORTANT there is a contribution by {RESUME_COPYWRITER_AGENT_NAME} at least once.
        - If RESPONSE is by {RESUME_COPYWRITER_AGENT_NAME} and contains the exact phrase "{RESUME_REVIEW_COMPLETE_KEYWORD}", make {INTERVIEW_PREPARATION_AGENT_NAME} the final participant.
        - After {INTERVIEW_PREPARATION_AGENT_NAME} has provided interview preparation guidance, do not select any more participants and indicate "{PROCESS_COMPLETE}" instead.

        RESPONSE:
        {{{{$lastmessage}}}}
        """,
    )

### Define the conditions under which the conversation ends

In [ ]:
def create_termination_strategy(agents, final_agent, maximum_iterations):
    """
    Create a chat termination strategy that terminates when the final agent is reached.
    params:
        agents: List of agents to trigger termination evaluation
        final_agent: The agent that should trigger termination
        maximum_iterations: Maximum number of iterations before termination
    """
    class CompletionTerminationStrategy(TerminationStrategy):
        async def should_agent_terminate(self, agent, history):
            """Terminate if the last actor is the Responder Agent."""
            logging.getLogger(__name__).debug(history[-1])
            return (agent.name == final_agent.name)

    return CompletionTerminationStrategy(agents=agents,
                                            maximum_iterations=maximum_iterations)

### Initialise the multi-agent chat

In [ ]:
agent_group_chat = AgentGroupChat(
    agents = [web_job_research_agent, resume_copywriter_agent, resume_reviewer_agent, interview_preparer_agent],
    selection_strategy=KernelFunctionSelectionStrategy(
            function=selection_function,
            kernel=kernel,
            result_parser=lambda result: str(result.value[0]) if result.value is not None else RESUME_REVIEWER_AGENT_NAME,
            agent_variable_name="agents",
            history_variable_name="lastmessage",
        ),
    termination_strategy=create_termination_strategy(
                agents=[web_job_research_agent, resume_copywriter_agent, resume_reviewer_agent, interview_preparer_agent],
                final_agent=interview_preparer_agent,
                maximum_iterations=MAXIMUM_CHAT_ITERATIONS
            ),
)

**Note:** Ignore INFO messages like **semantic_kernel.contents.chat_history - INFO - Could not parse prompt <...> as xml, treating as text, error was:**
this is a known "**feature**" explained in the link below:

https://github.com/microsoft/semantic-kernel/issues/10425

### Execute the multi-agent chat conversation, with tracing

In [ ]:
conversation_messages = []
conversation_messages.append({'role': 'user', 'name': 'user', 'content': "https://jobs.lever.co/AIFund/29e4750a-61c1-4195-9a11-7889577e3d6f"})

chat_history = [
    ChatMessageContent(
        role=AuthorRole(d.get('role')),
        name=d.get('name'),
        content=d.get('content')
    ) for d in filter(lambda m: m['role'] in ("system", "developer", "assistant", "user"), conversation_messages)
]

await agent_group_chat.add_chat_messages(chat_history)

tracer = get_tracer(__name__)
with tracer.start_as_current_span("AgenticChat"):
    async for _ in agent_group_chat.invoke():
        pass

response = list(reversed([item async for item in agent_group_chat.get_chat_messages()]))


### Output the conversation

In [ ]:
for i, res in enumerate(response):
    print(f"\n--- Response {i} ---")
    print(f"Role: {res.role.value}")
    print(f"Name: {res.name if hasattr(res, 'name') and res.name else 'None'}")
    print(f"Content: {res.content}")